In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point
import osmnx as ox

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
nodes = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Vias_Transporte/nodes_AMG.geojson', driver='GeoJSON')
print(nodes.shape)
nodes.head(2)

(172090, 5)


,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3.0,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,POINT (-103.31665 20.70065)


In [3]:
edges = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Vias_Transporte/edges_speed_AMG.geojson', driver='GeoJSON')
print(edges.shape)
edges.head(2)

(414774, 16)


,id,u,v,key,osmid,oneway,length,grade,grade_abs,bridge,junction,service,tunnel,walkspeed,time_min,geometry
0,-1.922695e+09,1.680913e+09,1.680913e+09,0.0,155732288,False,45.603,0.022,0.022,None,None,None,None,3.703559,0.738797,"LINESTRING (-103.25834 20.68905, -103.25844 20..."
1,-1.922105e+09,4.343498e+09,4.343498e+09,0.0,436461417,False,141.661,0.042,0.042,None,None,None,None,3.453176,2.461404,"LINESTRING (-103.19724 20.66496, -103.19742 20..."


In [22]:
edges = edges[~edges.u.isna()].copy()
print(edges.shape)
edges.head(2)

(414773, 16)


,id,u,v,key,osmid,oneway,length,grade,grade_abs,bridge,junction,service,tunnel,walkspeed,time_min,geometry
0,-1.922695e+09,1.680913e+09,1.680913e+09,0.0,155732288,False,45.603,0.022,0.022,None,None,None,None,3.703559,0.738797,"LINESTRING (-103.25834 20.68905, -103.25844 20..."
1,-1.922105e+09,4.343498e+09,4.343498e+09,0.0,436461417,False,141.661,0.042,0.042,None,None,None,None,3.453176,2.461404,"LINESTRING (-103.19724 20.66496, -103.19742 20..."


In [4]:
museo = nodes.loc[nodes.osmid==999999].copy()

In [23]:
nodes_analysis = nodes.set_index('osmid')
edges_analysis = edges.set_index(['u','v','key'])

In [24]:
nodes_dist = aup.calculate_distance_nearest_poi(museo, nodes_analysis, 
                edges_analysis, 'museo', 'osmid', wght='time_min')

In [25]:
nodes_dist.rename(columns={'dist_museo':'time_museo'}, inplace=True)

In [26]:
nodes_dist.to_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Analisis/nodes_time_AMG_NewConect.geojson', driver='GeoJSON')

In [ ]:
hex_gdf = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Area_Estudio/Hex_AMG.geojson', driver='GeoJSON')
print(hex_gdf.shape)
hex_gdf.head(2)

In [35]:
nodes_dist.reset_index(inplace=True)
nodes_dist = nodes_dist.set_crs("EPSG:4326")
hex_gdf = hex_gdf.set_crs("EPSG:4326")
col_name = 'time_museo'
hex_dist = aup.group_by_hex_mean(nodes_dist, hex_gdf, 8, col_name)

In [36]:
hex_gdf = hex_gdf.merge(hex_dist[['hex_id_8',col_name]], 
on='hex_id_8')

In [37]:
hex_gdf

,hex_id_8,CVEGEO,geometry,time_museo
0,88498cd6b5fffff,14001,"POLYGON ((-102.89959 20.75324, -102.89907 20.7...",0.000000
1,88498c99edfffff,14001,"POLYGON ((-102.94697 20.69376, -102.94243 20.6...",0.000000
2,88498c993bfffff,14001,"POLYGON ((-102.94697 20.69376, -102.94645 20.6...",0.000000
3,88498c8a07fffff,14001,"POLYGON ((-103.00449 20.78276, -103.00956 20.7...",0.000000
4,88498cd6edfffff,14001,"POLYGON ((-102.88529 20.70739, -102.88477 20.7...",0.000000
...,...,...,...,...
5200,88498c98d5fffff,14124,"POLYGON ((-103.05428 20.69941, -103.04974 20.7...",784.943054
5201,88498c815bfffff,14124,"POLYGON ((-103.11272 20.74897, -103.10817 20.7...",702.796309
5202,8849aa6e89fffff,14124,"POLYGON ((-102.93286 20.55354, -102.93234 20.5...",990.833477
5203,88498c9901fffff,14124,"POLYGON ((-102.93423 20.66349, -102.92968 20.6...",982.486308


In [40]:
hex_gdf.loc[hex_gdf.time_museo>0].to_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Analisis/hex_time_AMG.geojson', driver='GeoJSON')

In [27]:
mun_gdf = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Limites_Politicos/AMG_Jalisco12_UTM_v1.shp', driver='ESRIShapefile')

In [28]:
mun_gdf = mun_gdf.to_crs("EPSG:4326")

In [29]:
nodes = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Analisis/nodes_time_AMG_NewConect.geojson', driver='GeoJSON')
print(nodes.shape)
nodes.head(2)

(171941, 6)


,osmid,x,y,street_count,time_museo,geometry
0,28751344,-103.306309,20.711533,3.0,143.855790,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,141.796499,POINT (-103.31665 20.70065)


In [30]:
res = 9
hex_gdf = aup.create_hexgrid(mun_gdf, res)

In [31]:
nodes_dist = nodes.copy()
#nodes_dist.reset_index(inplace=True)
nodes_dist = nodes_dist.set_crs("EPSG:4326")
hex_gdf = hex_gdf.set_crs("EPSG:4326")
col_name = 'time_museo'
hex_dist = aup.group_by_hex_mean(nodes_dist, hex_gdf, res, col_name)

In [32]:
hex_gdf = hex_gdf.merge(hex_dist[[f'hex_id_{res}',col_name]], 
on=f'hex_id_{res}')

In [33]:
hex_gdf

,hex_id_9,geometry,time_museo
0,89498c96c73ffff,"POLYGON ((-103.37556 20.66096, -103.37434 20.6...",149.073717
1,89498c941c3ffff,"POLYGON ((-103.32476 20.69282, -103.32354 20.6...",140.622572
2,89498c941a7ffff,"POLYGON ((-103.34297 20.70119, -103.34175 20.7...",104.370034
3,89498c96b07ffff,"POLYGON ((-103.37745 20.70576, -103.37623 20.7...",63.463963
4,89498c94463ffff,"POLYGON ((-103.30791 20.67082, -103.30669 20.6...",195.435285
...,...,...,...
26178,89498c8044fffff,"POLYGON ((-103.13929 20.73746, -103.13806 20.7...",647.221179
26179,89498c82533ffff,"POLYGON ((-103.20409 20.69309, -103.20287 20.6...",0.000000
26180,89498c9ae2bffff,"POLYGON ((-103.04988 20.64926, -103.04866 20.6...",728.294851
26181,89498c91b47ffff,"POLYGON ((-103.12662 20.67459, -103.12540 20.6...",0.000000


In [34]:
hex_gdf.loc[hex_gdf.time_museo>0].to_file(f'/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Analisis/hex_{res}_time_AMG_NewConect.geojson', driver='GeoJSON')